#### Loading Data

In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(ggpubr))
suppressPackageStartupMessages(library(dplyr))
load("environment/real_datasets/GSE100505/mapDf.RData")
load("environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/seMuscle.RData")
load("environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/trMeanCorUPGMANorm.RData")
save_dir <- "environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees"
suppressPackageStartupMessages(source("brain_simulation_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/tree_analysis/tree_filter.R"))

#### Reading all the nwk files for the trees

In [2]:
dir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/real_datasets/GSE100505/term_out/mode_gcbias=True/posttype=gibbs_npost=100_tf=100"

treesCons0 <- read.tree(file.path(dir, "no_threshold0", "cluster_nwk.txt"))
txpsCons0 <- rownames(seMuscle)[as.numeric(unlist(lapply(treesCons0, function(tr) tr$tip)))+1]
                                                         
treesCons <- read.tree(file.path(dir, "no_threshold", "cluster_nwk.txt"))
txpsCons <- rownames(seMuscle)[as.numeric(unlist(lapply(treesCons, function(tr) tr$tip)))+1]
                                                        
treesMeanInf0 <- read.tree(file.path(dir, "no_threshold_mean_inf_0", "group_nwk.txt"))
txpsMeanInf0 <- rownames(seMuscle)[as.numeric(unlist(lapply(treesMeanInf0, function(tr) tr$tip)))+1]
                                                            
treesThr <- read.tree(file.path(dir, "threshold", "cluster_nwk.txt"))
txpsThr <- rownames(seMuscle[as.numeric(unlist(lapply(treesThr, function(tr) tr$tip)))+1])

groupsThr <- read.delim(file.path(dir, "old", "SRR5758702", "clusters.txt"), header = F)
txpsOld <- unlist(sapply(strsplit(groupsThr$V1, split=",", fixed=T), function(x) x[2:length(x)]))
                         
txpsCor <- trMeanCorUPGMANorm$tip

In [5]:
gse <- suppressWarnings(summarizeToGene(seMuscle))
gse <- fishpond::computeInfRV(gse)
save(gse, file=file.path(save_dir, "gse.RData"))

In [3]:
se <- labelKeep(seMuscle)

In [19]:
length(txpsThr)
length(txpsCor)
length(txpsOld)
length(txpsMeanInf0)
length(txpsCons0)
length(txpsCons)
length(rownames(se)[mcols(se)[["keep"]]])

[1] 17939

[1] 68062

[1] 7792

[1] 98091

[1] 97360

[1] 89429

[1] 45305

In [4]:
txpsAll <- Reduce(union,list(txpsCons0, txpsCons, txpsThr, txpsOld, txpsMeanInf0, txpsCor))
length(txpsAll)
seMuscleScaled <- scaleInfReps(seMuscle)

[1] 102091

#### Consensus Tree

In [25]:
treeCons <- mergeTree(treesCons, se = mapDf)
mb <- mergeLeaves(treeCons, seMuscle[txpsAll,])
treeCons <- mb[["tree"]]
yConsNS <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yConsNS) != treeCons$tip)
yAggConsNS <- prepSwish(treeCons, seMuscle[treeCons$tip,])
yAggConsNS <- fishpond::computeInfRV(yAggConsNS, meanVariance=F)
yAggCons <- prepSwish(treeCons, seMuscleScaled[treeCons$tip,])
infReps <- assays(yAggCons)[grep("infRep", assayNames(yAggCons))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggConsNS)[["LFC"]] <- getLog2FC(infReps, colData(yAggCons)[["condition"]])
rm(yAggCons)
gc()
save(yAggConsNS, file=file.path(save_dir, "yAggConsNS.RData"))
save(treeCons, file=file.path(save_dir, "treeCons.RData"))

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,10905698,582.5,19639040,1048.9,19639040,1048.9
Vcells,958000694,7309.0,2009783006,15333.5,1674609654,12776.3


In [26]:
yAggConsNS
treeCons

class: SummarizedExperiment 
dim: 171495 12 
metadata(7): tximetaInfo quantInfo ... txdbInfo infRepsScaled
assays(105): counts abundance ... mean variance
rownames(171495): ENSMUST00000161745.2 ENSMUST00000066175.9 ... 171494
  171495
rowData names(2): meanInfRV LFC
colnames(12): SRR5758624 SRR5758625 ... SRR5758703 SRR5758702
colData names(2): names condition


Phylogenetic tree with 102091 tips and 69404 internal nodes.

Tip labels:
  ENSMUST00000161745.2, ENSMUST00000066175.9, ENSMUST00000172439.1, ENSMUST00000134708.1, ENSMUST00000223737.1, ENSMUST00000185966.1, ...

Unrooted; includes branch lengths.

#### Consensus Tree 0

In [28]:
treeCons0 <- mergeTree(treesCons0, se = mapDf)
mb <- mergeLeaves(treeCons0, seMuscle[txpsAll,])
treeCons0 <- mb[["tree"]]
yConsNS0 <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yConsNS0) != treeCons0$tip)
yAggConsNS0 <- prepSwish(treeCons0, seMuscle[treeCons0$tip,])
yAggConsNS0 <- fishpond::computeInfRV(yAggConsNS0, meanVariance=F)
yAggCons0 <- prepSwish(treeCons0, seMuscleScaled[treeCons0$tip,])
infReps <- assays(yAggCons0)[grep("infRep", assayNames(yAggCons0))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggConsNS0)[["LFC"]] <- getLog2FC(infReps, colData(yAggCons0)[["condition"]])
rm(yAggCons0)
gc()
save(yAggConsNS0, file=file.path(save_dir, "yAggConsNS0.RData"))
save(treeCons0, file=file.path(save_dir, "treeCons0.RData"))

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,10918878,583.2,19639040,1048.9,19639040,1048.9
Vcells,1354389501,10333.2,2411819607,18400.8,2411705146,18399.9


#### Mean Inf Tree 0

In [30]:
treeMeanInf0 <- mergeTree(treesMeanInf0, se = mapDf)
mb <- mergeLeaves(treeMeanInf0, seMuscle[txpsAll,])
treeMeanInf0 <- mb[["tree"]]
yMeanInfNS0 <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yMeanInfNS0) != treeMeanInf0$tip)
yAggMeanInfNS0 <- prepSwish(treeMeanInf0, seMuscle[treeMeanInf0$tip,])
yAggMeanInfNS0 <- fishpond::computeInfRV(yAggMeanInfNS0, meanVariance=F)
yAggMeanInf0 <- prepSwish(treeMeanInf0, seMuscleScaled[treeMeanInf0$tip,])
infReps <- assays(yAggMeanInf0)[grep("infRep", assayNames(yAggMeanInf0))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggMeanInfNS0)[["LFC"]] <- getLog2FC(infReps, colData(yAggMeanInf0)[["condition"]])
rm(yAggMeanInf0)
gc()
save(yAggMeanInfNS0, file=file.path(save_dir, "yAggMeanInfNS0.RData"))
save(treeMeanInf0, file=file.path(save_dir, "treeMeanInf0.RData"))

[1] "Missing txps 4000"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,10919805,583.2,19639040,1048.9,19639040,1048.9
Vcells,1491927859,11382.6,2411819607,18400.8,2411705146,18399.9


[1] 0

Warning message in rm(yAggCons0):
"object 'yAggCons0' not found"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,10925121,583.5,19639040,1048.9,19639040,1048.9
Vcells,1995837506,15227.1,3473196233,26498.4,3213583920,24517.7


#### Correlation

In [32]:
mb <- mergeLeaves(trMeanCorUPGMANorm, seMuscle[txpsAll,])
treeCor <- mb[["tree"]]
rm(mb)
gc()
yAggCorNS <- prepSwish(treeCor, seMuscle[treeCor$tip,])
yAggCorNS <- fishpond::computeInfRV(yAggCorNS, meanVariance=F)
yAggCor <- prepSwish(treeCor, seMuscleScaled[treeCor$tip,])

infReps <- assays(yAggCor)[grep("infRep", assayNames(yAggCor))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggCorNS)[["LFC"]] <- getLog2FC(infReps, colData(yAggCor)[["condition"]])
rm(yAggCor)
gc()
save(yAggCorNS, file=file.path(save_dir, "yAggCorNS.RData"))
save(treeCor, file=file.path(save_dir, "treeCor.RData"))

[1] "Missing txps 34029"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,10925015,583.5,19639040,1048.9,19639040,1048.9
Vcells,1996541499,15232.5,3473196233,26498.4,3213583920,24517.7


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,10925860,583.6,19639040,1048.9,19639040,1048.9
Vcells,2211422604,16871.9,4167915479,31798.7,3661073694,27931.8


#### Threshold

In [33]:
treeThr <- mergeTree(treesThr, se = mapDf)
mb <- mergeLeaves(treeThr, seMuscle[txpsAll,])
treeThr<- mb[["tree"]]
yThrNS <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yThrNS) != treeThr$tip)

yAggThrNS <- prepSwish(treeThr, seMuscle[treeThr$tip,])
yAggThrNS <- fishpond::computeInfRV(yAggThrNS, meanVariance=F)

yAggThr <- prepSwish(treeThr, seMuscleScaled[treeThr$tip,])

infReps <- assays(yAggThr)[grep("infRep", assayNames(yAggThr))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggThrNS)[["LFC"]] <- getLog2FC(infReps, colData(yAggThrNS)[["condition"]])

rm(yAggThr)
gc()

save(yAggThrNS, file=file.path(save_dir, "yAggThrNS.RData"))
save(treeThr, file=file.path(save_dir, "treeThr.RData"))

[1] "Missing txps 84152"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,10926729,583.6,19639040,1048.9,19639040,1048.9
Vcells,2349009769,17921.6,4167915479,31798.7,3661073694,27931.8


[1] 0

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,10927593,583.6,19639040,1048.9,19639040,1048.9
Vcells,2434666228,18575.1,4167915479,31798.7,3724641431,28416.8


#### Terminus

In [8]:
suppressPackageStartupMessages(source("old_terminus_helper.R"))
mtxps <- setdiff(txpsAll, txpsOld)

yTermThrNS <- seMuscle[txpsAll,]
groupsClust <- parseClustFile(file.path(dir, "old", "SRR5758702", "clusters.txt"), yTermThrNS)
mInds <- seq(nrow(yTermThrNS) + length(groups))
yAggTermThrNS <- prepOSwish(yTermThrNS, mInds, groupsClust)
rownames(yAggTermThrNS)[length(txpsAll)+1:length(groupsClust)] <- as.character(length(txpsAll)+1:length(groupsClust))
yAggTermThrNS <- fishpond::computeInfRV(yAggTermThrNS)
save(yAggTermThrNS, file = file.path(save_dir, "yAggTermThrNS.RData") )

In [34]:
#### NGenes
nNodes <- length(treeCor$tip)+1:treeCor$Nnode
desc <- Descendants(treeCor, nNodes)
genesCor <- mclapply(desc, function(txps) unique(mapDf[treeCor$tip.label[txps],1]), mc.cores=1)
save(genesCor, file=file.path(save_dir, "genesCor.RData"))

In [8]:
# mb <- mergeLeaves(trMeanCorUPGMANorm, seMuscle[txpsAll,])
# treeCorNorm <- mb[["tree"]]
# rm(mb)
# gc()
yAggCorNormNS <- prepSwish(treeCorNorm, seMuscle[treeCorNorm$tip,])
yAggCorNormNS <- computeInfRV(yAggCorNormNS, meanVariance=F)
# yAggCor <- prepSwish(treeCor, se[treeCor$tip,])
# mcols(yAggCor) <- mcols(yAggCorNS)
save(yAggCorNormNS, file="environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/yAggCorNormNS.RData")
# save(yAggCor, file="environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/yAggCor.RData")
save(treeCorNorm, file="environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/treeCorNorm.RData")

In [37]:
#### NGenes
nNodes <- length(treeCons$tip)+1:treeCons$Nnode
desc <- Descendants(treeCons, nNodes)
genesCons <- mclapply(desc, function(txps) unique(mapDf[treeCons$tip.label[txps],1]), mc.cores=1)
save(genesCons, file=file.path(save_dir, "genesCons.RData"))

In [38]:
#### NGenes
nNodes <- length(treeCons0$tip)+1:treeCons0$Nnode
desc <- Descendants(treeCons0, nNodes)
genesCons0 <- mclapply(desc, function(txps) unique(mapDf[treeCons0$tip.label[txps],1]), mc.cores=1)
save(genesCons0, file=file.path(save_dir, "genesCons0.RData"))

In [39]:
#### NGenes
nNodes <- length(treeMeanInf0$tip)+1:treeMeanInf0$Nnode
desc <- Descendants(treeMeanInf0, nNodes)
genesMeanInf0 <- mclapply(desc, function(txps) unique(mapDf[treeMeanInf0$tip.label[txps],1]), mc.cores=1)
save(genesMeanInf0, file=file.path(save_dir, "genesMeanInf0.RData"))

In [46]:
#### NGenes
nNodes <- length(treeThr$tip)+1:treeThr$Nnode
desc <- Descendants(treeThr, nNodes)
genesThr <- mclapply(desc, function(txps) unique(mapDf[treeThr$tip.label[txps],1]), mc.cores=1)
save(genesThr, file=file.path(save_dir, "genesThr.RData"))

In [50]:
table(sapply(genesThr,length))
table(sapply(genesCons,length))


    1     2     3     4     5     6     7     8     9    10    11    12    13 
 8969   747   226   123    87    70    61    36    31    26    23    24    21 
   14    15    16    17    18    19    20    21    22    23    24    25    26 
   20    17    16    14    15    12    10    11    13    10    10    11     8 
   27    28    29    30    31    32    33    34    35    36    37    38    39 
    8     8     8     7     8     7     7     7     7     7     8     7     7 
   40    41    42    43    44    45    46    47    48    49    50    51    52 
    7     6     6     7     6     6     7     6     6     6     7     5     4 
   53    54    55    56    57    58    59    60    61    62    63    64    65 
    5     5     4     4     4     4     4     4     4     4     4     4     4 
   66    67    68    69    70    71    72    73    74    75    76    77    78 
    4     3     3     2     2     2     2     2     2     2     1     2     1 
   79    80    81    82    83    84    85    86    


    1     2     3     4     5     6     7     8     9    10    11    12    13 
60561  5426  1515   517   258   189   115    74    63    65    48    34    32 
   14    15    16    17    18    19    20    21    22    23    24    25    26 
   26    19    26    21    16    17    16    13    14    15    11    11    12 
   27    28    29    30    31    32    33    34    35    36    37    38    39 
   10    10    10     9     8     9     8     8    10     7     7     7     8 
   40    41    42    43    44    45    46    47    48    49    50    51    52 
    7     6     6     4     4     5     5     4     4     4     5     3     3 
   53    54    55    56    57    58    59    60    61    62    63    64    65 
    4     3     3     3     3     2     3     3     3     3     4     3     4 
   66    67    68    69    70    71    72    73    74    75    76    77    78 
    2     1     1     1     1     2     2     1     1     1     1     2     1 
   79    80    81    82    83    84    85    86    

In [48]:
treeThr
treeCons


Phylogenetic tree with 102091 tips and 10928 internal nodes.

Tip labels:
  ENSMUST00000093217.8, ENSMUST00000136822.2, ENSMUST00000161745.2, ENSMUST00000106298.9, ENSMUST00000136801.7, ENSMUST00000122860.1, ...

Unrooted; includes branch lengths.


Phylogenetic tree with 102091 tips and 69404 internal nodes.

Tip labels:
  ENSMUST00000161745.2, ENSMUST00000066175.9, ENSMUST00000172439.1, ENSMUST00000134708.1, ENSMUST00000223737.1, ENSMUST00000185966.1, ...

Unrooted; includes branch lengths.